In [ ]:
# 必要なライブラリをインストール
# 一度やれば~/.local以下にインストールされるので再実行する必要はないです
# 
# !pip install transformers accelerate langchain tiktoken sentence_transformers faiss-gpu boto3

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer



In [ ]:
# オブジェクトストレージにアクセスするための設定
# 使用するオブジェクトストレージに合わせて変更すること
# キーをベタ書きしてるので良い子はまねしないでね

accesskey = "LpkzPlUyAwDO2CQHE0XnW307GdqSJtms"
secretkey = "bUewoPAJnoKI2FhzJPYQnnFkLG-v382z"
endpoint = "http://10.38.76.10"
bucket = "langchain-bucket"


In [ ]:
##############################################################
# 事前に作成しておいたベクトルDBをオブジェクトストレージからロードする

import boto3
import pickle

s3_client = boto3.client( 
    "s3",
    aws_access_key_id=accesskey ,
    aws_secret_access_key=secretkey,
    endpoint_url=endpoint
)

response = s3_client.get_object(
    Bucket=bucket, 
    Key="vectorstore.pkl"
)

body = response['Body'].read()
vectorstore = pickle.loads(body)

In [ ]:
###################################################
# ベクトル検索

import time

question = "Nutanixでイレイジャーコーディングが適しているのはどのような場合ですか？"

start = time.time()
# 質問に対して、データベース中の類似度上位3件を抽出。質問の文章はこの関数でベクトル化され利用される
docs = vectorstore.similarity_search(question, k=3)
elapsed_time = time.time() - start
print(f"処理時間[s]: {elapsed_time:.2f}")
for i in range(len(docs)):
    print(docs[i])

In [ ]:
##########################################################
# LLMをロードする。

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

assert transformers.__version__ >= "4.34.1"

model = AutoModelForCausalLM.from_pretrained("cyberagent/calm2-7b-chat", device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("cyberagent/calm2-7b-chat")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:

#################################################################
# RAG のためのLangChainのインタフェース準備
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.8,
    streamer=streamer,
)
llm = HuggingFacePipeline(pipeline=pipe)



In [ ]:
# プロンプトの準備

DEFAULT_SYSTEM_PROMPT = "参考情報を元に、ユーザーからの質問に簡潔に正確に答えてください。\n"
text = "参考情報：\n{context}\n\nユーザからの質問：\n{question}"
template = "{bos_token}{system}{prompt}".format(
    bos_token=tokenizer.bos_token,
    system=DEFAULT_SYSTEM_PROMPT,
    prompt=text,
)

rag_prompt_custom = PromptTemplate(
    template=template, input_variables=["context", "question"]
)


In [ ]:
# チェーンの準備
chain = load_qa_chain(llm, chain_type="stuff", prompt=rag_prompt_custom)

In [ ]:
###################################################
# 生成
#######################################################

# まずはRAGなしでLLMに質問してみる

# RAG なしの場合
# 質問内容のみを入力として、文章生成
inputs = template.format(context="", question=question)
start = time.time()
output = llm(inputs)
elapsed_time = time.time() - start
print("RAGなし")
print(f"処理時間[s]: {elapsed_time:.2f}")
print(f"出力内容：\n{output}")
print(f"トークン数: {llm.get_num_tokens(output)}")



In [ ]:


# RAG ありの場合
start = time.time()
# ベクトル検索結果の上位3件と質問内容を入力として文章生成
inputs = {"input_documents": docs, "question": question}
output = chain.run(inputs)
elapsed_time = time.time() - start
print("RAGあり")
print(f"処理時間[s]: {elapsed_time:.2f}")
print(f"出力内容：\n{output}")
print(f"トークン数: {llm.get_num_tokens(output)}")


In [ ]:
# メモリを解放

del model, tokenizer, pipe, llm, chain


In [ ]:
torch.cuda.empty_cache()